In [1]:
#pip install tensorflow==1.14

In [2]:
# import torch
# torch.cuda.is_available()

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp")

mnist_train_images = mnist.train.images
mnist_train_labels = mnist.train.labels

mnist_test_images = mnist.test.images
mnist_test_labels = mnist.test.labels

class MNISTdataset(Dataset):
  
  def __init__(self, X: np.ndarray, y: np.ndarray):
    self.X = np.reshape(X, (X.shape[0], 28, 28))
    self.y = y.astype(np.long)
    self.preprocessing = transforms.Compose([
      transforms.ToTensor(), # first, convert image to PyTorch tensor
      transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
    ])
    
  def __len__(self):
    return self.X.shape[0]
  
  def __getitem__(self, item: int):
    Xcurr, ycurr = self.X[item], self.y[item]
    Xcurr = self.preprocessing(Xcurr)
    return Xcurr, ycurr

In [4]:
BATCH_SIZE = 128

mnist_train = MNISTdataset(mnist_train_images, mnist_train_labels)
mnist_test = MNISTdataset(mnist_test_images, mnist_test_labels)

dl_train = DataLoader(dataset=mnist_train, batch_size=BATCH_SIZE, 
                      num_workers=4, drop_last=True, shuffle=True)
dl_test = DataLoader(dataset=mnist_test, batch_size=BATCH_SIZE, 
                     num_workers=4, drop_last=False, shuffle=False)

In [ ]:
idx = np.random.choice(np.arange(0, mnist_train_images.shape[0]))
img = mnist_train_images[idx]
img = np.reshape(img, (28, 28))

fig = plt.figure(figsize = (12,12)) 
ax = fig.add_subplot(111)
ax.imshow(img, cmap='gray')
width, height = img.shape
thresh = img.max()/2.5
for x in range(width):
    for y in range(height):
        val = round(img[x][y],2) if img[x][y] !=0 else 0
        ax.annotate(str(val), xy=(y,x),
                    horizontalalignment='center',
                    verticalalignment='center',
                    color='white' if img[x][y]<thresh else 'black')

In [ ]:
num_row, num_col = 1, 10
f,subplots = plt.subplots(num_row, num_col, sharex='col', sharey='row')

for i in range(num_col):
    X, y = iter(dl_train).next()
    X_img = X[y == i]
    idx = np.random.choice(np.arange(0, X_img.shape[0]))
    
    X_img = X_img[idx]
    X_img = X_img.detach().cpu().numpy()
    X_img = X_img.squeeze()
    
    subplots[i].imshow(X_img, cmap='gray',
                       interpolation='nearest', 
                       aspect='auto')
    subplots[i].set_title(f'Digit {i}', fontweight="bold")
    subplots[i].grid(b=False)
    subplots[i].axis('off')

f.set_size_inches(22.5, 4.5)

In [7]:
import torch 

num_epochs = 10
num_fin = 784
num_classes = 10
num_hidden = 128
learning_rate = 0.01
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
import torch
import torch.nn as nn
from torch.optim import SGD

"""
    Write HERE your multi layer perceptron (MLP), with one hidden layer 
    characterised by num_hidden neurons, activated by one OF the following
    activation functions: sigmoid, relu, leaky_relu (see the torch.nn package
    for details about act. functions)

    Here a list of functions that you may use: 
      - nn.Sequential
      - nn.Linear
      - nn.Sigmoid
      - nn.ReLU
      - torch.argmax
      - optim.SGD, optim.Adam
      - nn.CrossEntropyLoss
      
    OPTIONALLY: 
    i) Add more than just one hidden layer. 
    ii) Put dropout and batch normalization layers to respectively improve
    the generalization capabilities and speedup training procedure.
"""

class MLP(nn.Module):
  
  def __init__(self, num_fin: int, num_hidden: int, num_classes: int):
    super(MLP, self).__init__()
    self.net = nn.Sequential(
                    nn.Linear(num_fin, num_hidden),
                    nn.ReLU(),
                    nn.Linear(num_hidden, num_classes)
    )
  
  def forward(self, x: torch.Tensor):
    return self.net(torch.flatten(x, 1))
  

def eval_acc(mlp: nn.Module, data_loader: torch.utils.data.DataLoader, 
             device: torch.device):
  
  correct = 0
  total = 0
  
  with torch.no_grad():
    for x, y in data_loader:
      x, y = x.to(device), y.to(device)
      y_pred = model(x)
      y_pred_discr = torch.argmax(y_pred, dim=1)
      acc = torch.sum((y_pred_discr == y).float()) 
      correct += acc
      total += y_pred.size(0)
  
  return correct/total

In [ ]:
model = MLP(num_fin, num_hidden, num_classes).to(device)
loss_fun = nn.CrossEntropyLoss().to(device)
opt = SGD(model.parameters(), learning_rate)

try:
  
    for i in range(num_epochs):
   
      print(f"Epoch {i} train acc.: {eval_acc(model, dl_train, device):.3f} "
                        f"test acc.: {eval_acc(model, dl_test, device):.3f}")
  
      for x, y in dl_train:
        x, y = x.to(device), y.to(device)

        opt.zero_grad()
        y_pred = model(x)
        loss = loss_fun(y_pred, y)
        loss.backward()
        opt.step()  
        
except KeyboardInterrupt:
    pass

In [ ]:
num_row, num_col = 2, 6
f,subplots = plt.subplots(num_row, num_col, sharex='col', sharey='row')

X_batch, Y_batch = next(iter(dl_test))
X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)

with torch.no_grad():
  Y_batch_pred = model(X_batch)

for i in range(num_row):
    for j in range(num_col):
      
        idx = np.random.choice(np.arange(0, X_batch.shape[0]))
        
        img = X_batch[idx].detach().cpu().numpy()
        
        target, pred = Y_batch[idx], Y_batch_pred[idx]
        target, pred = target.item(), torch.argmax(pred).item()
        target, pred = int(target), int(pred)
        is_correct = target == pred
                  
        subplots[i,j].imshow(img.squeeze(), cmap="gray", interpolation='nearest', 
                             aspect="auto")
        
        title = f"GT: {target} \n Pred: {pred}"
        color_title = 'green' if is_correct else 'red'
        subplots[i,j].set_title(title, color=color_title, fontweight="bold")
        subplots[i,j].grid(b=False)

f.set_size_inches(13.5, 7.5)